# Gastroscopy Demand Modelling by Age Bands – All Pathways

This notebook focuses exclusively on modelling gastroscopy demand in England, segmented into six clinically relevant age bands. We leverage aggregated counts from the National Endoscopy Database (NED) alongside 2021 mid-year population estimates to derive per-capita utilisation rates and fit predictive models.

| Age Band | UK Population (2021) | Estimated Gastroscopies (NED) | Rate per 1,000 |
|----------|----------------------|-------------------------------|----------------|
| 18–39    | 21,328,743           | 61,944                        | 2.90           |
| 40–49    | 8,441,035            | 54,297                        | 6.43           |
| 50–59    | 9,214,338            | 76,092                        | 8.26           |
| 60–69    | 7,310,130            | 77,621                        | 10.62          |
| 70–79    | 5,835,403            | 74,945                        | 12.84          |
| 80+      | 3,347,594            | 38,471                        | 11.49          |

**Data Source**

The figures presented here are drawn from the original research article by Beaton et al., “Diagnostic yield from symptomatic gastroscopy in the UK: British Society of Gastroenterology analysis using data from the National Endoscopy Database.” This cross-sectional study analysed 382 370 diagnostic gastroscopies performed between 1 March 2019 and 29 February 2020. Mixed-effect logistic regression models (with endoscopist as a random effect and symptoms, age and sex as fixed effects) were used to estimate adjusted positive predictive values for cancer and Barrett’s oesophagus, guiding the utilisation rates reported above.  


In [2]:
# ---- Step 0: Import Packages ----

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score
)

# Set plot style
sns.set(style="whitegrid")
plt.rcParams["figure.dpi"] = 120

In [3]:
# ---- Step 1: Load Raw Input Files and Preview ----

# Define paths
BASE_PATH = (
    "/Users/rosstaylor/Downloads/Research Project/Code Folder/"
    "nhs-diagnostics-dids-eda/nhs-dids-explorer/data/processed"
)

ENDO_PATH = os.path.join(BASE_PATH, "endo_master.csv")
MODALITY_COUNTS_PATH = os.path.join(BASE_PATH, "modality_procedure_counts_by_age_band_2024.csv")
POPULATION_PATH = os.path.join(BASE_PATH, "south_west_population_2024.csv")

# Load CSV files
endo_master_df = pd.read_csv(ENDO_PATH)
modality_counts_df = pd.read_csv(MODALITY_COUNTS_PATH)
population_df = pd.read_csv(POPULATION_PATH)

# Preview key dataframes
for name, df in {
    "Endoscopy Master": endo_master_df,
    "Modality Counts (2024)": modality_counts_df,
    "Population (2024)": population_df,
}.items():
    print(f"\n{name} — shape: {df.shape}")
    print(df.head())



Endoscopy Master — shape: (7174, 30)
  icb_code                icb_name  lsoa_code  \
0      11X  NHS SOMERSET ICB - 11X  E01029316   
1      11X  NHS SOMERSET ICB - 11X  E01029134   
2      11X  NHS SOMERSET ICB - 11X  E01029331   
3      11X  NHS SOMERSET ICB - 11X  E01029052   
4      11X  NHS SOMERSET ICB - 11X  E01029205   

                             nhs_region site_code  \
0  Abdominal structure (body structure)     RH5A8   
1  Abdominal structure (body structure)     RA7C2   
2  Abdominal structure (body structure)     RH5A8   
3  Abdominal structure (body structure)     RH504   
4  Abdominal structure (body structure)     RH504   

                         site_name provider_code  \
0   MUSGROVE PARK HOSPITAL (RH5A8)           RH5   
1  WESTON GENERAL HOSPITAL (RA7C2)           RA7   
2   MUSGROVE PARK HOSPITAL (RH5A8)           RH5   
3   GREENFIELDS DAY CENTRE (RH504)           RH5   
4   GREENFIELDS DAY CENTRE (RH504)           RH5   

                                   

In [5]:
# ----------------------------
# Set up band definitions
# ----------------------------
# NB: bins are *right-inclusive* (17, 39, 49 … fall in the lower band)
BINS   = [-np.inf, 17, 39, 49, 59, 69, 79, np.inf]
LABELS = ['0–17', '18–39', '40–49', '50–59', '60–69', '70–79', '80+']

RATE_PER_1K = {          # from your table
    '18–39': 2.90,
    '40–49': 6.43,
    '50–59': 8.26,
    '60–69': 10.62,
    '70–79': 12.84,
    '80+'  : 11.49,
    # Optional: assume zero for under-18s unless you have a value
    '0–17' : 0.00,
}

# ----------------------------
#  Helper to label ages
# ----------------------------
def add_age_band(df, age_col='age', new_col='age_band'):
    """
    Add a categorical age-band column to *df*.
    df:       DataFrame with an age column (int or float).
    age_col:  name of that column.
    new_col:  destination column for the labels.
    """
    df[new_col] = pd.cut(df[age_col], bins=BINS, labels=LABELS, right=True)
    return df

# ----------------------------
# Example with Population (2024)
# ----------------------------

population_df = add_age_band(population_df)

# ── option A: coerce as soon as you create the column ──────────────
population_df['rate_per_1k'] = (
    population_df['age_band']
      .map(RATE_PER_1K)          # lookup
      .astype(float)             # <-- force numeric
)
# expected exam count per single year of age
population_df['expected_exams'] = population_df['population'] * population_df['rate_per_1k'] / 1000

# ----------------------------
# Aggregate to the band level
# ----------------------------
band_summary = (
    population_df
    .groupby('age_band', observed=True)
    .agg(population=('population', 'sum'),
         rate_per_1k=('rate_per_1k', 'first'),
         expected_exams=('expected_exams', 'sum'))
    .reset_index()
    .sort_values('age_band', key=lambda s: s.map({l:i for i,l in enumerate(LABELS)}))
)

print(band_summary)


  age_band  population  rate_per_1k  expected_exams
0     0–17     1112406         0.00         0.00000
1    18–39     1528575         2.90      4432.86750
2    40–49      694057         6.43      4462.78651
3    50–59      821965         8.26      6789.43090
4    60–69      700436        10.62      7438.63032
5    70–79      612458        12.84      7863.96072
6      80+      358312        11.49      4117.00488


In [8]:
# -------------------------------------------------------
# 0.  MAKE SYNTHETIC LSOA POPULATION BY SINGLE YEAR OF AGE
# -------------------------------------------------------
np.random.seed(42)   # reproducible “random” numbers

lsoas = [f"E010000{i:02d}" for i in range(1, 6)]          # 5 fake LSOA codes
rows  = []

for lsoa in lsoas:
    for age in range(0, 85):                              # ages 0-84 inclusive
        pop = np.random.poisson(lam=60)                   # ~60 people each age
        rows.append({"lsoa_code": lsoa, "age": age, "population": pop})

synthetic_df = pd.DataFrame(rows)

# -------------------------------------------------------
# 1.  ADD AGE-BAND & RATE, THEN EXPECTED EXAMS
# -------------------------------------------------------
synthetic_df = add_age_band(synthetic_df, age_col="age")

synthetic_df["rate_per_1k"] = (
    synthetic_df["age_band"]
      .map(RATE_PER_1K)
      .astype(float)
)

synthetic_df["expected_exams"] = (
    synthetic_df["population"] * synthetic_df["rate_per_1k"] / 1_000
)

# -------------------------------------------------------
# 2.  SUMMARY 1 – BY LSOA & BROAD AGE BAND
# -------------------------------------------------------
band_summary_all = (
    synthetic_df
      .groupby(["lsoa_code", "age_band"], observed=True)
      .agg(population     = ("population", "sum"),
           rate_per_1k    = ("rate_per_1k", "first"),
           expected_exams = ("expected_exams", "sum"))
      .reset_index()
      .sort_values(["lsoa_code", "age_band"])
)

print("=== Population & expected exams by LSOA × age-band ===")
print(band_summary_all.head(20))      # show a snippet

# -------------------------------------------------------
# 3.  SUMMARY 2 – TOTAL EXPECTED EXAMS PER LSOA
# -------------------------------------------------------
lsoa_totals = (
    band_summary_all
      .groupby("lsoa_code", observed=True)
      .agg(population       = ("population", "sum"),
           expected_exams   = ("expected_exams", "sum"))
      .reset_index()
)

print("\n=== Total expected gastroscopies per LSOA ===")
print(lsoa_totals)


=== Population & expected exams by LSOA × age-band ===
    lsoa_code age_band  population  rate_per_1k  expected_exams
0   E01000001     0–17        1043         0.00         0.00000
1   E01000001    18–39        1314         2.90         3.81060
2   E01000001    40–49         626         6.43         4.02518
3   E01000001    50–59         611         8.26         5.04686
4   E01000001    60–69         583        10.62         6.19146
5   E01000001    70–79         617        12.84         7.92228
6   E01000001      80+         309        11.49         3.55041
7   E01000002     0–17        1099         0.00         0.00000
8   E01000002    18–39        1314         2.90         3.81060
9   E01000002    40–49         628         6.43         4.03804
10  E01000002    50–59         575         8.26         4.74950
11  E01000002    60–69         586        10.62         6.22332
12  E01000002    70–79         595        12.84         7.63980
13  E01000002      80+         309        11.49  

In [10]:
# -------------------------------------------------------------------
# A.  Minimal helper (paste once, or import if you've saved it)
# -------------------------------------------------------------------
def screening_demand_45_74_simple(
    df,
    *,
    positivity="avg",              # "1.5", "2.5", "avg"  or float like 0.018
    lsoa_col="lsoa_code",
    age_col="age",
    pop_col="population",
    uptake=0.52,                   # average kit-return rate
    prefix="scr_"
):
    """Return screening & colonoscopy workload per LSOA for ages 45-74."""
    pos_map = {"1.5": 0.015, "2.5": 0.025, "avg": 0.020, "average": 0.020}
    pos_rate = float(pos_map.get(str(positivity).lower(), positivity))

    w = df[df[age_col].between(45, 74, inclusive="both")]

    invited = (
        w.groupby(lsoa_col, as_index=False)[pop_col]
         .sum()
         .rename(columns={pop_col: prefix + "invited_total"})
    )
    invited[prefix + "kits_returned"] = invited[prefix + "invited_total"] * uptake
    invited[prefix + "abnormal_kits"] = invited[prefix + "kits_returned"] * pos_rate
    invited[prefix + "colonoscopies"] = invited[prefix + "abnormal_kits"]
    return invited

# -------------------------------------------------------------------
# B.  Apply to your synthetic single-year table
# -------------------------------------------------------------------
scr_demand = screening_demand_45_74_simple(
    synthetic_df,
    positivity="2.5"        # ← choose "1.5", "2.5", "avg", or a float
)

print("=== Screening demand (ages 45-74) per LSOA ===")
print(scr_demand)

# -------------------------------------------------------------------
# C.  Combine with gastroscopy expectation already in lsoa_totals
# -------------------------------------------------------------------
combined = (
    lsoa_totals               # from your earlier step
      .merge(scr_demand, on="lsoa_code")
      .sort_values("lsoa_code")
)

print("\n=== Gastroscopy vs screening workload per LSOA ===")
print(combined)


=== Screening demand (ages 45-74) per LSOA ===
   lsoa_code  scr_invited_total  scr_kits_returned  scr_abnormal_kits  \
0  E01000001               1838             955.76             23.894   
1  E01000002               1771             920.92             23.023   
2  E01000003               1776             923.52             23.088   
3  E01000004               1752             911.04             22.776   
4  E01000005               1761             915.72             22.893   

   scr_colonoscopies  
0             23.894  
1             23.023  
2             23.088  
3             22.776  
4             22.893  

=== Gastroscopy vs screening workload per LSOA ===
   lsoa_code  population  expected_exams  scr_invited_total  \
0  E01000001        5103        30.54679               1838   
1  E01000002        5106        30.01167               1771   
2  E01000003        4957        29.05512               1776   
3  E01000004        4927        29.59022               1752   
4  E01000